# Weather-Based Plant Recommendation Engine

Name: Zihan

### Workflow Overview

This notebook implements the core logic for a weather-based plant recommendation system. The end-to-end process is as follows:

1.  **Setup and Data Loading:**
    * Import necessary Python libraries (`requests`, `pandas`, `numpy`, `json`, `random`).
    * Load the pre-processed plant dataset from `Table13_GeneralPlantListforRecommendation.csv` into a pandas DataFrame.
    * Another version is that directly access the data from MySQL.
    * The `sunlight` column, originally a JSON string, is parsed back into a Python list for easier processing.

2.  **Weather Data Fetching and Aggregation:**
    * A function, `get_and_aggregate_weather_data`, is defined to handle interactions with the Open-Meteo API.
    * Given a specific `latitude` and `longitude`, this function fetches a 16-day forecast for six key weather variables: minimum/maximum temperature, sunshine duration, max UV index, precipitation sum, and mean relative humidity.
    * It then aggregates this daily data into a summarized dictionary of six key metrics (e.g., `extreme_min_temp`, `avg_sunshine_duration`), handling potential `null` values and rounding the results to two decimal places.

3.  **Core Matching Logic Definition:**
    * The main recommendation "brain" is a function called `is_plant_suitable`.
    * This function takes the aggregated weather dictionary and a single plant's data (a DataFrame row) as input.
    * It applies a series of sophisticated rules to determine suitability:
        * **Temperature Survival:** Compares the forecast's extreme minimum temperature against the plant's absolute minimum survival temperature.
        * **Sunlight Needs:** Checks if the average sunshine duration and UV index match the plant's requirements for "full sun" or "part shade".
        * **Watering Needs:** Matches the average daily precipitation with the plant's preference for frequent, average, or minimal watering.
        * **Drought Tolerance:** Uses the average relative humidity to assess if a non-drought-tolerant plant can survive in a dry-air environment.

4.  **Execution and Output:**
    * The main script sets the target coordinates.
    * It calls the weather function to get the summarized forecast.
    * It applies the `is_plant_suitable` function to every plant in the DataFrame to filter out unsuitable options.
    * The resulting list of suitable plant IDs is then **randomly shuffled** to ensure variety in the presentation.
    * Finally, the notebook outputs the two key results: the summarized weather information and the randomized list of recommended plant IDs.

## Step 1 (Local Version): Import Libraries, Setup and Load Plant Data
First, we import all necessary libraries and load the Table13 we created earlier. An important step is to convert the sunlight column from JSON string back to Python list format for subsequent logic processing.

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import random 

# Set Table13 file path
TABLE13_PATH = "Table13.csv"

# Load plant data
try:
    df_plants = pd.read_csv(TABLE13_PATH)
    # Key step: convert sunlight column from string back to list
    df_plants['sunlight'] = df_plants['sunlight'].apply(json.loads)
    print("Plant data Table13 loaded and prepared successfully!")
    print(f"Loaded {len(df_plants)} plants.")
    df_plants.head()
except FileNotFoundError:
    print(f"Error: Plant data file not found, please check if path is correct: {TABLE13_PATH}")

植物数据 Table13 加载并准备成功！
共加载 1008 种植物。


## Step 1 (DB Version): Load Plant Data from MySQL Database

In [ ]:
# ==============================================================================
# Step 1: Import Libraries, Connect to Database and Load Plant Data
# ==============================================================================

# Import required libraries
import requests
import pandas as pd
import numpy as np
import json
import random
import mysql.connector # <-- New: Import MySQL connection library
from mysql.connector import Error

# --- 1. Database Connection Configuration ---
# Note: In production environment, do not hardcode passwords in code.
db_config = {
    'host': 'database-plantx.cqz06uycysiz.us-east-1.rds.amazonaws.com',
    'user': 'zihan',
    'password': '2002317Yzh12138.',
    'database': 'FIT5120_PlantX_Database',
    'use_pure': True,
    'charset': 'utf8mb4'
}

# --- 2. Load Data from Database ---
try:
    # Establish database connection
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("Successfully connected to MySQL database.")
        
        # Define SQL query statement
        query = "SELECT * FROM Table13_GeneralPlantListforRecommendation;"
        
        # Use pandas read_sql function to directly read query results into DataFrame
        df_plants = pd.read_sql(query, connection)
        
        print(f"Successfully loaded {len(df_plants)} plants from database.")

except Error as e:
    print(f"Error occurred while loading data from MySQL: {e}")
    # Create an empty DataFrame to avoid errors in subsequent cells
    df_plants = pd.DataFrame() 

finally:
    # Close database connection
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL database connection closed.")

# --- 3. Data Post-processing (same as before) ---
if not df_plants.empty:
    # Key step: convert sunlight column from string back to list
    df_plants['sunlight'] = df_plants['sunlight'].apply(json.loads)
    
    # Convert 0/1 values from database to boolean True/False to ensure correct logic
    df_plants['drought_tolerant'] = df_plants['drought_tolerant'].astype(bool)
    
    print("\nPlant data DataFrame is ready!")
    display(df_plants.head()) # In Jupyter, display() is more visually appealing than print()
else:
    print("\nFailed to load plant data, subsequent steps may not execute properly.")

成功连接到MySQL数据库。


C:\Users\zyyin1\AppData\Local\Temp\ipykernel_6608\3323652273.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_plants = pd.read_sql(query, connection)


成功从数据库加载 1008 种植物。
MySQL数据库连接已关闭。

植物数据 DataFrame 已准备就绪！


,general_plant_id,sunlight,watering,drought_tolerant,absolute_min_temp_c
0,398,"[full sun, part shade]",Average,True,-17.8
1,399,"[full sun, part shade]",Average,False,-23.3
2,400,"[full sun, part shade]",Average,True,-28.9
3,401,"[Full sun, part shade]",Average,False,-23.3
4,402,"[full sun, part shade]",Average,True,-28.9


## Step 2: Define Data Fetching and Aggregation Function
This function is responsible for calling the Open-Meteo API, fetching weather data, and aggregating it according to the pre-designed logic. It pays special attention to handling potential null values at the end of the data.

In [ ]:
def get_and_aggregate_weather_data(latitude, longitude):
    """
    根据经纬度获取未来16天天气预报，并聚合成我们需要的6个核心指标。
    """
    # 构建API请求URL
    api_url = (
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}"
        "&daily=precipitation_sum,sunshine_duration,uv_index_max,temperature_2m_max,"
        "temperature_2m_min,relative_humidity_2m_mean&timezone=auto&forecast_days=16"
    )
    
    # 发起API请求
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # 如果请求失败 (如 404, 500)，则会抛出异常
        weather_data = response.json()['daily']
    except requests.exceptions.RequestException as e:
        print(f"API请求失败: {e}")
        return None
    
    # --- 开始聚合数据 ---
    # 定义一个辅助函数来清洗列表中的null值
    def clean_list(data_list):
        return [item for item in data_list if item is not None]

    # 清洗所有数据列表
    temp_min_list = clean_list(weather_data['temperature_2m_min'])
    temp_max_list = clean_list(weather_data['temperature_2m_max'])
    sunshine_list = clean_list(weather_data['sunshine_duration'])
    uv_list = clean_list(weather_data['uv_index_max'])
    precipitation_list = clean_list(weather_data['precipitation_sum'])
    humidity_list = clean_list(weather_data['relative_humidity_2m_mean'])
    
    # Calculate 6 aggregated metrics
    aggregated_weather = {
        'extreme_min_temp': np.min(temp_min_list) if temp_min_list else None,
        'extreme_max_temp': np.max(temp_max_list) if temp_max_list else None,
        'avg_sunshine_duration': (np.mean(sunshine_list) / 3600) if sunshine_list else None, # seconds -> hours
        'avg_max_uv_index': np.mean(uv_list) if uv_list else None,
        'avg_daily_precipitation': np.mean(precipitation_list) if precipitation_list else None,
        'avg_relative_humidity': np.mean(humidity_list) if humidity_list else None
    }
    
    # --- [Modification] Before returning, round all floats to 2 decimal places ---
    formatted_weather = {
        key: round(value, 2) if isinstance(value, float) else value
        for key, value in aggregated_weather.items()
    }
    
    return formatted_weather

## Step 3: Define Core Matching Logic Function
这个函数是整个推荐系统的“大脑”。它将提前设计好的所有匹配规则代码化，判断单个植物是否符合聚合后的天气条件。

In [ ]:
def is_plant_suitable(agg_weather, plant_row):
    """
    Based on aggregated weather data and plant attributes, determine if the plant is suitable for planting.
    Returns True (suitable) or False (not suitable).
    """
    # Rule 1: Survival baseline check (hard filter)
    if agg_weather['extreme_min_temp'] < plant_row['absolute_min_temp_c']:
        return False

    # Rule 2: Sunlight requirement check (duration + intensity)
    sun_duration = agg_weather['avg_sunshine_duration']
    uv_index = agg_weather['avg_max_uv_index']
    sunlight_needs = plant_row['sunlight']
    
    sun_duration_ok = False
    if 'full sun' in sunlight_needs and sun_duration >= 6: sun_duration_ok = True
    if 'part shade' in sunlight_needs or 'part sun/part shade' in sunlight_needs and (3 <= sun_duration < 6): sun_duration_ok = True
    if 'full shade' in sunlight_needs and sun_duration < 3: sun_duration_ok = True
    
    if not sun_duration_ok: return False
    
    # Intensity optimization check
    if uv_index > 8 and sunlight_needs == ['part shade']: return False # UV too strong, not suitable for partial shade only
    if uv_index < 3 and sunlight_needs == ['full sun']: return False # UV too weak, cannot meet full sun requirements

    # Rule 3: Watering requirement check
    precipitation = agg_weather['avg_daily_precipitation']
    watering_needs = plant_row['watering']
    
    if watering_needs == 'Frequent' and precipitation < 3: return False
    if watering_needs == 'Minimal' and precipitation > 5: return False
    if watering_needs == 'Average' and not (1 <= precipitation <= 8): return False

    # Rule 4: Drought tolerance check
    humidity = agg_weather['avg_relative_humidity']
    is_drought_tolerant = plant_row['drought_tolerant']
    
    if not is_drought_tolerant and humidity < 40: return False

    # If all checks pass
    return True

## Step 4: Execute All Processes and Output Final Results
We specify a latitude and longitude, call functions to get and aggregate weather data, then apply matching functions to filter plants, and finally print the two results we need.

### Test Locations for Plant Recommendation

To better test the recommendation system, here are several locations in Melbourne and regional Victoria with different geographical and climatic conditions. This allows for testing the matching logic's performance in various scenarios (e.g., coastal, inland, high-altitude).

| Location | Area/Climate Notes | Latitude | Longitude |
| :--- | :--- | :--- | :--- |
| **Melbourne CBD** | Melbourne City Centre, urban heat island effect | `-37.8136` | `144.9631` |
| **Geelong** | City of Geelong, Victoria's second-largest city, on the western bay | `-38.1471` | `144.3603` |
| **Mornington** | Mornington Peninsula, coastal area, mild climate | `-38.2167` | `145.0333` |
| **Olinda** | Dandenong Ranges, high-altitude area, generally cooler and wetter | `-37.8500` | `145.3667` |
| **Bendigo** | City of Bendigo, inland Victoria, generally drier, hot summers, cold winters | `-36.7570` | `144.2794` |
| **Mildura** | Mildura, northwestern Victoria, agricultural area, hot and dry climate | `-34.1847` | `142.1587` |

To use these samples, simply modify the `latitude` and `longitude` variables in the final execution cell and re-run it to see the recommendation results for different locations.

In [ ]:
# --- Step 1: Set target latitude and longitude (using Melbourne as example) ---
latitude = -37.8136
longitude = 144.9631

# # --- Step 1: Set target latitude and longitude (using Olinda in Dandenong Ranges as example) ---
# latitude = -37.8500
# longitude = 145.3667

In [ ]:
# --- Step 2: Get and aggregate weather data ---
print(f"Getting weather data for coordinates ({latitude}, {longitude})...")
aggregated_weather_info = get_and_aggregate_weather_data(latitude, longitude)

# --- Step 3: Filter suitable plants ---
if aggregated_weather_info and 'df_plants' in locals():
    # Use .apply method to apply matching function to each row
    is_suitable_series = df_plants.apply(
        lambda row: is_plant_suitable(aggregated_weather_info, row),
        axis=1
    )
    
    # Get filtered plant ID list
    suitable_plant_ids = df_plants[is_suitable_series]['general_plant_id'].tolist()
    
    # --- [Modification] New code: randomly shuffle ID list ---
    random.shuffle(suitable_plant_ids)
    
    # --- Step 4: Output final results ---
    print("\n" + "="*50)
    print("      16-Day Aggregated Weather Information      ")
    print("="*50)
    for key, value in aggregated_weather_info.items():
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")
    
    print("\n" + "="*50)
    print("      Plant ID List Meeting Planting Conditions      ")
    print("="*50)
    if suitable_plant_ids:
        print(suitable_plant_ids)
    else:
        print("Based on future weather, no particularly suitable plants were found.")
    print("="*50)

else:
    print("\nDue to weather data retrieval failure or plant data not loaded, recommendations cannot be made.")

正在获取经纬度 (-37.8136, 144.9631) 的天气数据...



      未来16天聚合天气信息      
extreme_min_temp: 6.40
extreme_max_temp: 28.30
avg_sunshine_duration: 8.89
avg_max_uv_index: 4.73
avg_daily_precipitation: 0.29
avg_relative_humidity: 67.13

      符合种植条件的植物ID列表      
[1174, 1167, 694, 1169, 1182, 1184, 1178, 725, 472, 1256, 979, 1339, 847, 1186, 540, 1337, 1151, 781, 975, 1014, 1152, 606, 1235, 1236, 728, 443, 985, 1223, 972, 833, 908, 721, 779, 1181, 1166, 1237, 803, 439, 613, 612, 986, 692, 1165, 1162, 1098, 610, 1175, 802, 607, 1096, 726, 1025, 617, 1176, 492, 1183, 618, 1171, 568, 1170, 848, 1153, 611, 651, 1173, 1168, 1021, 1179, 849]
